# Reads Files and Populates Error Images
1) Loop to read files in folde rone by one
2) Open image file and its filter file
3) Group failuremodes by color codes in filter, Red: Pimple, Green: Dent, Blue: Scrach
4) Find the center location of each color group and create square images based on that center and save image 

In [1]:
!pip install opencv-python
!pip install xlsxwriter 

DEPRECATION: Python 2.7 will reach the end of its life on January 1st, 2020. Please upgrade your Python as Python 2.7 won't be maintained after that date. A future version of pip will drop support for Python 2.7.


DEPRECATION: Python 2.7 will reach the end of its life on January 1st, 2020. Please upgrade your Python as Python 2.7 won't be maintained after that date. A future version of pip will drop support for Python 2.7.


In [2]:
import numpy as np
import cv2
import os
import matplotlib.pyplot as plt
import xlsxwriter 
import time as tm
#e1 = cv2.getTickCount() # To count time
from tqdm import tqdm



In [3]:
Temptale = cv2.imread("./ColorTemplate.tif",1)
Rwi,Cli,Colori =  Temptale.shape
#for i  in range(Rwi):
#    for j  in range(Cli):
#        print Temptale[i,j]

In [4]:
#Definitions
Images_Directory = "./Images/"
Filters_Directory = "./Filters/"
Marked_Directory = "./Marked/"
Directory_P = "./Pimples/"
Directory_D = "./Dents/"
Directory_S = "./Scratch/"


# Below color codes are based on RGB
PimpleColor = [237,28,36] # Red
DentColor = [34,177,76] # Green
ScratchColor = [0,162,232] #Ligth Blue

SizeFilter = 20
FilterError = 10

PopulationSize=50
# OpenCV follows BGR order, while matplotlib likely follows RGB order.


In [6]:
#workbook = xlsxwriter.Workbook('./hello.xlsx')
#worksheet = workbook.add_worksheet() 

for filename in os.listdir(Images_Directory):
    if filename.endswith(".tiff"):
        BaseName = filename[0:-5]
        img_bw = cv2.imread(Images_Directory+filename,0) #read in greyscale
        filt1 = cv2.imread(Filters_Directory+BaseName+".tiff",1) #read in color
        # OpenCV follows BGR order, while matplotlib likely follows RGB order.
        b1,g1,r1 = cv2.split(filt1)
        filt1_rgb = cv2.merge([r1,g1,b1])
        Rw,Cl,Not =  filt1.shape
        #plt.imshow(filt1_rgb)
        print "Image Size: " + str(Rw) + " " + str (Cl)
        tm.sleep(1)
        Pimple=1
        Dent=1
        Scrtch=1
        Unknown = 1
        Total = 0
        Prev_Pimple = [0,0]
        Prev_Dent = [0,0]
        Prev_Scrtch = [0,0]
        Prev_Unknown = [0,0]
        FaultLocations = []
        FaultLocations_Pimple = []
        FaultLocations_Scrtch = []
        FaultLocations_Dent = []
        
       
        
        #FindCenterPoints
        for i  in tqdm(range(Rw)):
            for j  in range(Cl):
                b,g,r = filt1[i,j]
                #worksheet.write(i, j, str([r,g,b])) 
                if not (b==g and b==r):
                    if Total>0:
                        Diff=np.sum(np.square(np.array(FaultLocations)-np.array([i,j])),1)
                        Diff_Bool = np.size(Diff[Diff<SizeFilter**2])
                    else:
                        Diff_Bool = False
                    if np.sum(np.square(np.array([r,g,b])-np.array(PimpleColor)))<FilterError**2 and not Diff_Bool:
                       #print "Found Pimple " + str(Pimple) + " -- "+ str(i) + "," + str (j) 
                        Prev_Pimple = [i,j]
                        FaultLocations_Pimple.append([i,j])
                        FaultLocations.append([i,j])
                        Pimple+=1
                        Total+=1
                    elif np.sum(np.square(np.array([r,g,b])-np.array(DentColor)))<FilterError**2 and not Diff_Bool:
                        #print "Found Dent " + str(Dent)+ " -- "+ str(i) + "," + str (j)
                        Prev_Dent = [i,j]
                        FaultLocations_Dent.append([i,j])
                        FaultLocations.append([i,j])
                        Dent +=1
                        Total+=1
                    elif np.sum(np.square(np.array([r,g,b])-np.array(ScratchColor)))<FilterError**2 and not Diff_Bool:
                        #print "Found Scratch " + str(Scrtch)+ " -- "+ str(i) + "," + str (j)
                        Prev_Scrtch = [i,j]
                        FaultLocations_Scrtch.append([i,j])
                        FaultLocations.append([i,j])
                        Scrtch+=1
                        Total+=1
                    #elif ((i-Prev_Unknown[0])**2+(j-Prev_Unknown[1])**2)>SizeFilter**2:
                    #    print "Found Unknown " + str(Unknown)+ " -- "+ str(i) + "," + str (j)
                    #    Prev_Unknown = [i,j]
                    #    FaultLocations.append([i,j])  
                    #    Unknown+=1
                    #    Total+=1
                    #    break 
        print "Total Found: " + str(Total)

        img_clr = cv2.imread(Images_Directory+filename,1) #read in greyscale
        for k in range(len(FaultLocations_Pimple)):
            cntr = FaultLocations_Pimple[k]
            cv2.rectangle(img_clr,(cntr[1]-PopulationSize,cntr[0]+PopulationSize),(cntr[1]+PopulationSize,cntr[0]-PopulationSize),(PimpleColor[2], PimpleColor[1],PimpleColor[0]),3)
            cutfig = img_bw[cntr[0]-PopulationSize:cntr[0]+PopulationSize,cntr[1]-PopulationSize:cntr[1]+PopulationSize].copy()
            cv2.imwrite(Directory_P+BaseName+"_"+str(k)+".tiff",cutfig)
        for k in range(len(FaultLocations_Dent)):
            cntr = FaultLocations_Dent[k]
            cv2.rectangle(img_clr,(cntr[1]-PopulationSize,cntr[0]+PopulationSize),(cntr[1]+PopulationSize,cntr[0]-PopulationSize),(DentColor[2], DentColor[1],DentColor[0]),3)
            cutfig = img_bw[cntr[0]-PopulationSize:cntr[0]+PopulationSize,cntr[1]-PopulationSize:cntr[1]+PopulationSize].copy()
            cv2.imwrite(Directory_D+BaseName+"_"+str(k)+".tiff",cutfig)
        for k in range(len(FaultLocations_Scrtch)):
            cntr = FaultLocations_Scrtch[k]
            cv2.rectangle(img_clr,(cntr[1]-PopulationSize,cntr[0]+PopulationSize),(cntr[1]+PopulationSize,cntr[0]-PopulationSize),(ScratchColor[2], ScratchColor[1],ScratchColor[0]),3)
            cutfig = img_bw[cntr[0]-PopulationSize:cntr[0]+PopulationSize,cntr[1]-PopulationSize:cntr[1]+PopulationSize].copy()
            cv2.imwrite(Directory_S+BaseName+"_"+str(k)+".tiff",cutfig)
        
        
        
        #Resize image 
        scale_percent = 25 # percent of original size
        width = int(img_clr.shape[1] * scale_percent / 100)
        height = int(img_clr.shape[0] * scale_percent / 100)
        dim = (width, height)
        resized = cv2.resize(img_clr, dim, interpolation = cv2.INTER_AREA)

        cv2.imwrite(Marked_Directory+BaseName+"_marked.tiff",resized)
    continue
#workbook.close() 

Image Size: 3000 4096


100%|██████████| 3000/3000 [00:19<00:00, 152.35it/s]


Total Found: 41
Image Size: 3000 4096


100%|██████████| 3000/3000 [00:20<00:00, 143.47it/s]


Total Found: 35
Image Size: 3000 4096


...

100%|██████████| 3000/3000 [00:21<00:00, 139.40it/s]


Total Found: 36
Image Size: 3000 4096


100%|██████████| 3000/3000 [00:21<00:00, 140.43it/s]


Total Found: 32


ValueError: need more than 0 values to unpack

In [ ]:
plt.imshow(img_clr)
